# MovieLens

##DUE APRIL 21, 2016

[MovieLens](http://www.movielens.org/) is a website where users can submit ratings for movies that they watch and receive recommendations for other movies they might enjoy. The data is collected and made publicly available for research. We will be working with a data set of 1 million user ratings of movies. You can find this data set and even larger ones at http://grouplens.org/datasets/movielens/.


## Reading in the Data

Note that the data consists of three data frames: one with information about the users, another containing the ratings, and yet another with information about the movies. See the readme file (/data/movielens/README) for more information.

In [1]:
import pandas as pd

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('/data/movielens/users.dat', sep='::', header=None,
                      names=unames, engine="python")

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('/data/movielens/ratings.dat', sep='::', header=None,
                        names=rnames, engine="python")

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('/data/movielens/movies.dat', sep='::', header=None,
                       names=mnames, engine="python")

In [9]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
...,...,...,...,...
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518


## Question 1

Develop a way to rank movies. Why is it not a good idea to simply sort the movies by average rating? (You may want to try calculating this first.) Then, explain your methodology and use it to produce a list of the Top 10 movies of all time.

In [3]:
#ratings

In [4]:
# YOUR CODE HERE
#raise NotImplementedError()

# ratings_movie_id_avg = ratings.groupby(["movie_id" ]).mean()#.get(["rating"])
# #This would calculate the mean of each column, including userid and timestamp for ea movieid
# ratings_movie_id_avg




#Another way to rank movies:
# A 'top' movie is a movie with not only THOUSANDS of ratings, but on average, people gave it a 5 star rating.

num_ratings_per_movie = ratings.groupby(["movie_id"])[['rating']].count()
num_ratings_per_movie = num_ratings_per_movie.ix[num_ratings_per_movie['rating'] >= 2000 ] # get movies with >= # of ratings
num_ratings_per_movie = num_ratings_per_movie.reset_index() # call this after groupby
num_ratings_per_movie = num_ratings_per_movie.merge( ratings, on= 'movie_id')

top_movies = pd.merge(num_ratings_per_movie,users, on = "user_id")
top_movies = top_movies.ix[ top_movies['rating_y'] == 5].groupby('movie_id')[['rating_y']].mean()
top_movies.columns = ['average_rating']
top_movies.head(n=10)




# avg_ratings = ratings.ix[ratings['rating'] >= 4, :].groupby('movie_id')[ ['rating'] ].mean()
# # don't use get('rating') as that will use all cols when calling .mean() on it
# avg_ratings

,average_rating
movie_id,
1,5
110,5
260,5
296,5
318,5
356,5
480,5
527,5
589,5


It doesn't make sense to sort the movies by its average ratings alone because there may be some movies where only a few people gave it a rating. A movie with a rating defined by only a few reviewers is not a good rating because a few reviewers is not reprsentative of the entire audience at all.

The table outputted shows the top 10 movies of all time.

## Question 2

Restrict to movies with at least 200 ratings. For each movie, calculate the difference between the average male rating and the average female rating. Based on these differences between average male ratings and average female ratings, what movies were the most male-friendly? What movies were the most female-friendly?

In [5]:
# YOUR CODE HERE
#raise NotImplementedError()


num_ratings_per_movie = ratings.groupby(["movie_id"])[['rating']].count()
num_ratings_per_movie = num_ratings_per_movie.ix[num_ratings_per_movie['rating'] >= 200]
num_ratings_per_movie = num_ratings_per_movie.reset_index()
num_ratings_per_movie = num_ratings_per_movie.merge( ratings, on= 'movie_id')
#num_ratings_per_movie
#next step: merge num_ratings_per_movie_200 with original ratings table and then filter out.


data_merged_rate = pd.merge(num_ratings_per_movie,users, on = "user_id")

# # inner join:
data_merged_rate['Avg Female Rating'] = data_merged_rate.ix[ data_merged_rate['gender'] == 'F' ].groupby(['movie_id']).mean()['rating_y']
data_merged_rate['Avg Female Rating'] = data_merged_rate['Avg Female Rating'].fillna(0)
data_merged_rate['Avg Male Rating'] = data_merged_rate.ix[ data_merged_rate['gender'] == 'M' ].groupby( ['movie_id'] ).mean()['rating_y']
data_merged_rate['Avg Male Rating'] = data_merged_rate['Avg Male Rating'].fillna(0)
# #['rating'] is necessary because I only want to add a column

data_merged_rate['Diff Between Females\' and Males\' Avg Ratings' ] = data_merged_rate['Avg Female Rating']\
     - data_merged_rate['Avg Male Rating']
data_merged_rate = data_merged_rate.sort_values(by = 'Diff Between Females\' and Males\' Avg Ratings' ,ascending = True)

#data_merged_rate

print(data_merged_rate.head(n = 3))
#for most male friendly
print("=========================================================================")
print(data_merged_rate.tail(n = 3))
# for most  female friendly

      movie_id  rating_x  user_id  rating_y  timestamp gender  age  \
1201      3744       421       19         4  978147373      M    1   
3760      1784      1424       65         5  977888204      M   35   
1194      3624       791       19         5  978146984      M    1   

      occupation    zip  Avg Female Rating  Avg Male Rating  \
1201          10  48073           3.494949         4.221300   
3760          12  55803           2.878788         3.555147   
1194          10  48073           2.944444         3.585227   

      Diff Between Females' and Males' Avg Ratings  
1201                                     -0.726351  
3760                                     -0.676359  
1194                                     -0.640783  
      movie_id  rating_x  user_id  rating_y  timestamp gender  age  \
2468      1721      1546       38         4  978044771      F   18   
203       2699      1367        8         5  978229347      M   25   
1088      1792       448       19         3 

Movies(movie_id) that were most male friendly : 3744, 1784, 3624

Movies(move_id) that were most female friendly: 1721, 2699, 1792

*note: If difference between females' and males' average ratings is positive, that means that on average, females gave a particular movie higher rating than males did (female_friendly). If difference is negative, males gave it higher ratings on average.

## Question 3

Calculate the average rating by genre. Note that a movie can belong to multiple genres. You will have to write some code that parses the `genres` column of the `movies` table. What genre had the highest average rating? What genre had the lowest?

In [6]:
# YOUR CODE HERE
#raise NotImplementedError()

# merge the movies to the ratings
data = movies.merge(ratings, on="movie_id")

# get all occurrences of genres in a list
genres = movies["genres"].str.cat(sep="|").split("|")
# get unique genres, convert back to list
genres = list(set(genres))

# add a boolean column to DataFrame for each genre
for genre in genres:
    data[genre] = data["genres"].str.contains(genre)

# multiply each genre column by the ratings column,
# then .sum() to get the total of ratings for that genre
total_rating = data[genres].multiply(data['rating'], axis=0).sum()
# adding the booleans should give the number of ratings for each genre
num_rating = data[genres].sum()
# divide to get the average rating
#(total_rating / num_rating).sort_values()
print((total_rating / num_rating).sort_values().tail())
print((total_rating / num_rating).sort_values().head())



# split_genres = movies[ 'genres'].str.cat(sep = '|') # returns a string
# #split_genres

# split_genres = split_genres.split("|") # must concat before doing this step
# split_genres

# distinct_genres_list = []

# #create list of distinct genres
# # get unique genres, convert back to list
# #enres = list(set(genres))

# for genre in split_genres:
 
    
#     if genre not in distinct_genres_list:
        
#         distinct_genres_list.append(genre)



# data_merged_genre = pd.merge(ratings, movies, on = 'movie_id')
# data_merged_genre = data_merged_genre.sort_index()


# for genre in distinct_genres_list:
    
#         data_merged_genre[genre] = data_merged_genre['genres'].str.contains(genre)
        
# data_merged_genre

# # multiply each genre column by the ratings column,
# # then .sum() to get the total of ratings for that genre
# total_rating = data_merged_genre[split_genres].multiply(data_merged_genre['rating'], axis=0).sum()
# # adding the booleans should give the number of ratings for each genre
# num_rating = data_merged_genre[genres].sum()
# # divide to get the average rating
# (total_rating / num_rating).sort_values()
        
# #.str.contains('Drama')
# #data_merged_genre['genres'].str.contains('Drama')
# #use .split?

# # data_merged_genre = data_merged_genre.groupby('genres' )[ ['rating'] ].mean()
# # print(data_merged_genre.sort_values(by = 'rating').tail())
# # print(data_merged_genre.sort_values(by = 'rating').head(n = 50))

Crime          3.708679
Drama          3.766332
War            3.893327
Documentary    3.933123
Film-Noir      4.075188
dtype: float64
Horror        3.215013
Children's    3.422035
Fantasy       3.447371
Sci-Fi        3.466521
Adventure     3.477257
dtype: float64


Ignoring mixed genres, the Animation genre had the highest average ratings. Also, ignoring mixed genres, the Children genre had
the lowest average ratings.

CORRECTION: Horror genre had lowest avg ratings. Film-Noi has highest.

## Question 4

Formulate a question of your own that you can answer using this MovieLens data. State clearly what your question is and what your findings are. Bonus points are available if you find something interesting!

**Tip:** You may find the `occupation` column of `users` to be a rich source of interesting questions. See the README file (/data/movielens/README) for information about how `occupation` is coded in the data.

In [7]:
# YOUR CODE HERE
#raise NotImplementedError()

num_ratings_by_occupation = users.merge(ratings, on = 'user_id').groupby('occupation')[['rating']].count()
num_ratings_by_occupation.columns = ['number of ratings']
num_ratings_by_occupation

,number of ratings
occupation,
0,130499
1,85351
2,50068
3,31623
4,131032
5,21850
6,37205
...,...
14,49109


Q: How many ratings did each group of people by occupation give? Which occupation had people that gave the most ratings? 

Users(people) with occupation 4 rated the most amount of movies as a group. One could speculate that those with occupation 4 were more willing to rate a movie that they watched. It is also possible that the movies those in occupation 4 chose to watch tended to provoke strong feelings.

## Submitting this Lab

Now, restart your kernel and re-run your entire notebook from beginning to end. Make sure there are no errors or bugs. When you have verified this, open the Terminal on JupyterHub and type 

`nbgrader submit Lab-04-14 --course dlsun`

to submit this lab.